In [25]:
import requests
import pandas as pd

api_token = '8a0b92957b57d4eea9cdf73c4d7436c09c82a38865b84e5b2304b7b5a0202b92'
coins_endpoint = f'https://whattomine.com/api/v1/coins?api_token={api_token}'
coins_data = requests.get(coins_endpoint).json()

coins = [ 'BTC', 'BCH', 'DOGE', 'LTC', 'KAS', 'RVN' ]
columns = [ 'block_time', 'block_reward', 'exchange_rate_btc', 'algorithm', 'nethash' ] 
index = []
data = []

for c in coins_data:
    if c['tag'] not in coins:
        continue
    index.append(c['tag'])
    data.append([ float(c['block_time']), c['block_reward'],  1 if c['tag'] == 'BTC' else c['exchanges'][0]['price'], c['algorithm'], c['nethash'] ])
df = pd.DataFrame(data, index=index, columns=columns)
df['block_reward_btc'] = df.block_reward * df.exchange_rate_btc
df['daily_emmisions_btc'] = 86400 / df.block_time * df.block_reward_btc
df

,block_time,block_reward,exchange_rate_btc,algorithm,nethash,block_reward_btc,daily_emmisions_btc
BCH,609.0000,3.125000,3.410623e-03,SHA-256,2649132618349513957,0.010658,1.512099
BTC,663.0000,3.158051,1.000000e+00,SHA-256,700315690625637858220,3.158051,411.546896
DOGE,63.0000,10000.000000,2.709000e-06,Scrypt,2909032600071622,0.027090,37.152000
KAS,0.9995,65.421518,9.975386e-07,kHeavyHash,1385449409073613806,0.000065,5.641327
LTC,150.0000,6.250000,1.071600e-03,Scrypt,2129993045490122,0.006698,3.857760
RVN,60.0000,2500.000000,1.534879e-07,KawPow,6833541749549,0.000384,0.552556


In [70]:
import requests
import pandas as pd

algorithms_response = requests.get('https://api2.nicehash.com/main/api/v2/public/simplemultialgo/info')
multialgo_data = algorithms_response.json()

algorithms = [ 'SCRYPT', 'SHA256ASICBOOST', 'KAWPOW', 'KHEAVYHASH']
columns = ['paying', 'speed']
index = []
data = []
for a in multialgo_data['miningAlgorithms']:
    algo = a['algorithm']
    if algo not in algorithms:
        continue
    index.append(algo)
    data.append([a['paying'], a['speed']])
algo_data_df = pd.DataFrame(data, index=index, columns=columns)

factors_response = requests.get('https://api2.nicehash.com/main/api/v2/mining/algorithms')
factors_data = factors_response.json()

columns = ['market_factor', 'mining_factor', 'price_factor']
index = []
data = []
for f in factors_data['miningAlgorithms']:
    algo = f['algorithm']
    if algo not in algorithms:
        continue
    index.append(algo)
    data.append([float(f['marketFactor']), float(f['miningFactor']), float(f['priceFactor'])])
factors_data_df = pd.DataFrame(data, index=index, columns=columns)
factors_data_df

sats2btc = 1e8
market_data_df = pd.DataFrame(index=index)
market_data_df['hashrate'] = algo_data_df['speed'] * factors_data_df['mining_factor'] / factors_data_df['market_factor']
market_data_df['paying'] = algo_data_df['paying'] * factors_data_df['price_factor'] / sats2btc

market_data_df


,hashrate,paying
SCRYPT,91.544926,0.030102
SHA256ASICBOOST,6.921981,0.568777
KAWPOW,0.879280,0.082559
KHEAVYHASH,55.708149,0.003920
